In [2]:
import time
import tkinter as tk
from heapq import heappush, heappop

# Simplified Dungeon Grid
dungeon = [
    ['S', '.', '.', '.','.','#', '.','.','.','.','.'],
    ['#', '#', '.', '.','.','.', '.','.','X','.','#'],
    ['.', '#', '.', '.','.','#', '.','.','.','.','.'],
    ['.', '#', '.', '.','.','#', '.','.','.','#','.'],
    ['.', '#', '.', '.','.','X', '.','.','.','X','.'],
    ['.', '#', 'X', '.','.','X', '.','.','.','X','.'],
    ['.', '#', '#', '#','#','X', '#','#','.','#','X'],
    ['.', '.', '.', '.','.','.', 'X','#','.','.','.'],
    ['.', '.', '.', '#','.','.', '#','.','.','.','.'],
    ['.', '.', '.', '#','.','.', '#','.','.','.','.'],
    ['.', '.', '.', '#','.','.', 'X','#','.','X','.'],
    ['.', '.', '.', '#','.','.', '#','#','.','.','X'],
    ['.', '.', '.', '#','.','.', '#','.','X','.','G']
]

start = (0, 0)
goal = (12, 10)
cell_size = 30

def heuristic(a, b):
    """Manhattan distance heuristic for grid."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    open_set = []
    heappush(open_set, (0 + heuristic(start, goal), 0, start))
    came_from = {start: None}
    g_score = {start: 0}
    explored = []
    all_visited = set()

    while open_set:
        _, current_g, current = heappop(open_set)
        if current not in all_visited:
            explored.append(current)
            all_visited.add(current)

        if current == goal:
            path = []
            while current:
                path.append(current)
                current = came_from[current]
            return path[::-1], explored

        for dx, dy in directions:
            nx, ny = current[0] + dx, current[1] + dy
            if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] not in ['#', 'X']:
                neighbor = (nx, ny)
                tentative_g_score = current_g + 1

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    g_score[neighbor] = tentative_g_score
                    f_score = tentative_g_score + heuristic(neighbor, goal)
                    heappush(open_set, (f_score, tentative_g_score, neighbor))
                    came_from[neighbor] = current

    return None, explored

def visualize_search(grid, path, explored):
    window = tk.Tk()
    window.title("Dungeon Explorer Visualization")
    canvas = tk.Canvas(window, width=len(grid[0]) * cell_size, height=len(grid) * cell_size + 50)
    canvas.pack()

    shadow_offset = 5  # Shadow offset for 3D effect

    def draw_cell(x, y, color, shadow=False):
        """Draws a single cell with or without shadow"""
        x1, y1 = x * cell_size, y * cell_size
        x2, y2 = x1 + cell_size, y1 + cell_size
        if shadow:
            canvas.create_rectangle(x1 + shadow_offset, y1 + shadow_offset, x2 + shadow_offset, y2 + shadow_offset,
                                    fill="gray", outline="red", width=2)
        canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="red")

    # Initialize all the cells
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            color = "white"
            if grid[i][j] == '#':
                color = "black"  # Obstacle
            elif grid[i][j] == 'S':
                color = "green"  # Start
            elif grid[i][j] == 'G':
                color = "red"  # Goal
            elif grid[i][j] == 'X':
                color = "orange"  # Trap

            draw_cell(j, i, color)

    # Add a text area for displaying time and cells explored
    info_label = tk.Label(window, text="", font=("Arial", 14), anchor="w", justify="left")
    info_label.pack()

    # Animate explored cells
    explored_idx = 0
    start_time = time.time()

    def animate_explored():
        nonlocal explored_idx
        if explored_idx < len(explored):
            x, y = explored[explored_idx]
            draw_cell(y, x, "lightblue", shadow=True)
            explored_idx += 1
            elapsed_time = time.time() - start_time
            info_label.config(
                text=f"Time Elapsed: {elapsed_time:.2f} seconds\nCells Explored: {explored_idx}")
            window.after(50, animate_explored)  # Redraw after 50ms to create an animation effect

    # Animate path cells
    path_idx = 0

    def animate_path():
        nonlocal path_idx
        if path_idx < len(path):
            x, y = path[path_idx]
            draw_cell(y, x, "yellow", shadow=True)
            path_idx += 1
            elapsed_time = time.time() - start_time
            info_label.config(
                text=f"Path Found in: {elapsed_time:.2f} seconds\nCells Explored: {len(explored)}\nPath Length: {len(path)}")
            window.after(150, animate_path)  # Redraw after 150ms to create an animation effect

    # Start the animation for explored cells first
    animate_explored()

    # Once the explored animation finishes, animate the path
    def start_path_animation():
        window.after(50 * len(explored), animate_path)  # Start path animation after a short delay

    window.after(50 * len(explored), start_path_animation)

    window.mainloop()

# Run A* and visualize
path, explored = a_star_search(dungeon, start, goal)
if path is None:
    print("No path found between start and goal!")
else:
    print(f"Path found: {path}")
visualize_search(dungeon, path, explored)


Path found: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 5), (1, 6), (1, 7), (2, 7), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (11, 9), (12, 9), (12, 10)]
